In [22]:
import hashlib

class MerkleTree:
    def __init__ (self, data):
        self.data = data
        self.tree = self._build_tree()

    def _build_tree(self):
        if len(self.data) % 2 != 0:
            self.data.append(self.data[-1]) # Duplicate the last element for an even number of elements

        tree = [hashlib.sha256(str(value).encode()).hexdigest() for value in self.data]

        while len(tree) > 1:
            tree = [hashlib.sha256((tree[i] + tree[i + 1]).encode()).hexdigest() for i in range(0,len(tree), 2)]

        return tree

    def get_root(self):
        return self.tree[0]

    def get_proof(self, index):
        if index < 0 or index >= len(self.data):
            raise ValueError("Index out of range")

        proof = []
        currentIndex = index
        for level in range(len(self.tree) // 2):
            isRightChild = currentIndex % 2
            siblingIndex = currentIndex + 1 if isRightChild else currentIndex - 1
            proof.append((isRightChild, self.tree[siblingIndex]))
            currentIndex = currentIndex // 2
        return proof

def verify_proof(root, index, value, proof):
    currentHash = hashlib.sha256(str(value).encode()).hexdigest()

    for isRightChild, siblingHash in proof:
        if isRightChild:
            currentHash = hashlib.sha256((siblingHash + currentHash).encode()).hexdigest()
        else:
            currentHash = hashlib.sha256((currentHash + siblingHash).encode()).hexdigest()
        return currentHash == root

# Example usage:
data = ["Transaction 1", "Transaction 2", "Transaction 3", "Transaction 4"]
merkle_tree = MerkleTree(data)
print("Merkle Tree Root:", merkle_tree.get_root())

# Verify proof for the first transaction
index_to_verify = 0
proof = merkle_tree.get_proof(index_to_verify)
result = verify_proof(merkle_tree.get_root(), index_to_verify, data[index_to_verify], proof)
print(f"Verification result for Transaction {index_to_verify + 1}: {result}")

Merkle Tree Root: 7e1182c5c00e379261503e757487cfa16cda7010f1ca3ae0115d5b78cfc07509
Verification result for Transaction 1: None


In [5]:
#Creation of Block.
def create_block(data, previous_hash=None): 
    block = {
'data': data,
'previous_hash' : previous_hash,
'hash' : hash(data + str(previous_hash))
}
    return block
block_data = "Some data for the block" 
previous_hash = "Previous Block's hash"
my_block = create_block(data=block_data, previous_hash=previous_hash)
print(f"Block data: {my_block['data']}") 
print(f"Previous hash: {my_block['previous_hash']}") 
print(f"Block hash: {my_block['hash']}")

Block data: Some data for the block
Previous hash: Previous Block's hash
Block hash: -8494754838577224203


In [6]:
#prac 3
# 1. Create a simple client class that generates public key and private key by using built-in python technique RSA (Rivest–Shamir–Adleman) algorithm and test it.
# 2. Create a transaction class to send and receive money and test it.
# 3. Create multiple transactions and display them.
# 4. Creating blockchain, a genesis block and execute it.
# 5. Create a mining function and test it.
# 6. Add blocks to the miner and dump the blockchain.

#1
import binascii
import Crypto
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5
class Client:
    def __init__(self):
        random = Crypto.Random.new().read
        self._private_key = RSA.generate(1024, random)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)
    @property
    def identity(self):
        return binascii.hexlify(self._public_key.exportKey(format="DER")).decode(
"ascii"
)
Dinesh = Client()
print("\n Public Key:",Dinesh.identity)
#print("Private Key:", Dinesh._private_key.export_key().decode("utf-8"))


 Public Key: 30819f300d06092a864886f70d010101050003818d0030818902818100eee52ca55c25d801b7c2ea3e3927a8b4f21c062f64da3527fcfebac17a5fe7216fa4713c1d1cd727fc2a076996027a841f1313d12c25610538f2f74e6c33db6fed1674732ffdbf9a19ab31dee218bd40508f1160f724a3c14fb42d02750157ad8722f446c7772a1c77145fe4d2db3b2b5de7618d0ff62d074b772aba599dc7630203010001


In [13]:
# 2. Create a transaction class to send and receive money and test it.
import binascii
import collections
import datetime
from Crypto.Hash import SHA
from Crypto.Signature import PKCS1_v1_5
class Transaction:
    def __init__(self, sender, recipient, value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()
    def to_dict(self):
        identity = "Genesis" if self.sender == "Genesis" else self.sender.identity
        return collections.OrderedDict(
        {
        "sender": identity,
        "recipient": self.recipient,
        "value": self.value,
        "time": self.time,
        }
)
    def sign_transaction(self):
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode("utf8"))
        return binascii.hexlify(signer.sign(h)).decode("ascii")
    def display_transaction(self):
        print("Sender:", self.sender)
        print("Recipient:", self.recipient)
        print("Value:", self.value)
        print("Time:", self.time)
Dinesh = Client()
Ramesh = Client()

t = Transaction(Dinesh, Ramesh.identity, 5.0)
print("\nTransaction Recipient:\n", t.recipient)
# print("\nTransaction Sender:\n", t.sender)
print("\nTransaction Value:\n", t.value)
signature = t.sign_transaction()
print("\nSignature:\n", signature)


Transaction Recipient:
 30819f300d06092a864886f70d010101050003818d0030818902818100b98361d6c9041dd1083605a287d2e3fe0698caf2e14e4e3dc8f6bee9161bf2243b703ca1fd088e44900f4dfba00a7469b8274015ee3e7685bb56906fa0ea85abd1408cc8021afc8328d1124aee0feca927edc5565d9932038686ec960d0ecd8c046685a9259bba80cbbb4bd483d1987c971293e39e6a4359af8c28ea0885a2f30203010001

Transaction Value:
 5.0

Signature:
 96003482a17cce47c9f543897e0aaf3d7bcde59c909989da68dae4aadd9278a3aa4dd83ea481403b23ee1c8009f6bc96a344c1ed0e7698d01e8fed82620773f6515a771133ce5790c872540aeedf0d8416d07d55250dde85cdd985db49984fe5006bbef629041f1b5d3de50cce6590047be2ceb5fc35b4155ec716fd71aa7875


In [14]:
# 3. Create multiple transactions and display them.
Dinesh = Client()
Ramesh = Client()

t = Transaction(Dinesh, Ramesh.identity, 5.0)
print("\nTransaction Recipient:\n", t.recipient)
# print("\nTransaction Sender:\n", t.sender)
print("\nTransaction Value:\n", t.value)
signature = t.sign_transaction()
print("\nSignature:\n", signature)
Dinesh = Client()
Ramesh = Client()
Seema = Client()
Vijay = Client()
t1 = Transaction(Dinesh, Ramesh.identity, 15.0)
t1.sign_transaction()
transactions = [t1]
t2 = Transaction(Dinesh, Seema.identity, 6.0)
t2.sign_transaction()
transactions.append(t2)
t3 = Transaction(Ramesh, Vijay.identity, 2.0)
t3.sign_transaction()
transactions.append(t3)
t4 = Transaction(Seema, Ramesh.identity, 4.0)
t4.sign_transaction()
transactions.append(t4)
for transaction in transactions:
    Transaction.display_transaction(transaction)
    print("--------------")


Transaction Recipient:
 30819f300d06092a864886f70d010101050003818d0030818902818100d5d793628ad0a4b7a501386a5f0deb3b31490aad16373bc86bf535734bf6de969f361d8feb45e64a39c90960ab521829a8c6255111218df7f369b18ea60a655b86328bcc367651d88e1b866f3e4f7f293a18dfcf08d06b4026219f1fe4c027d12dee5f4919e600477aa90430c901292c5a1323bc224c81dbbb236a7d127289e50203010001

Transaction Value:
 5.0

Signature:
 7e2de368fba70764b575d654583aaf07c1c4768e6d0830248327319ab32a463c0caf4263164da38825e662c69431f5e9f58e1d2e05afe4e6eb7bdf0888d7a8cec82eaadc410a56c39584383725d4fa688b160a215e9a13a1cd1e6fe13dc7f50d5fb58dfa283eefc9e45eea1f3c2df9a02bff92cb9119590cb7a2c5d9eb8edb7b
Sender: <__main__.Client object at 0x000001CC9FA147D0>
Recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100ced65ba47a5af44f34dcc35b7658f58d85e3ef05538aa23756ed685e0f50e6c387aa06b1882fcf7c55db13b64e4ddd796186868a7faca109f7047126515eec3eb01c300841fb744a87bd17667d12b3bb5f897a9db56536af01342e1ccf4f73699dafae58d08d1058eb5989b81faf623b6326e36

In [18]:
# 4. Creating blockchain, a genesis block and execute it.
class Block:
    def __init__(self, client):
        self.verified_transactions = []
        self.previous_block_hash = ""
        self.Nonce = ""
        self.client = client
    
def dump_blockchain(blocks):
        print(f"\nNumber of blocks in the chain: {len(blocks)}")
        for i, block in enumerate(blocks):
            print(f"block # {i}")
            for transaction in block.verified_transactions:
                Transaction.display_transaction(transaction)
                print("--------------")
        print("=====================================")
Dinesh = Client()
t0 = Transaction("Genesis", Dinesh.identity, 500.0)

block0 = Block(Dinesh)
block0.previous_block_hash = ""
NONCE = None

block0.verified_transactions.append(t0)
digest = hash(block0)
last_block_hash = digest

TPCoins = [block0]
dump_blockchain(TPCoins)    


Number of blocks in the chain: 1
block # 0
Sender: Genesis
Recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100d196036a970f866a3182ca6987c94d9923abfa9ff05627d62f4de731f9ef86baee1fb2215df6a98df8c9737bea2db74aad5f9c2fba3d7ff1de8a7eac2f8c1159bf2651ccb92ee218151ad79af935e87372c78ba086a9551282eea51bd3620d89fb50382c94821071b82b58f16130539125f667a0c0927501bec1684121822aaf0203010001
Value: 500.0
Time: 2024-04-01 08:16:09.026124
--------------


In [19]:
#Create a mining function and test it.
import hashlib
def sha256(message):
    return hashlib.sha256(message.encode("ascii")).hexdigest()
def mine(message, difficulty=1):
    assert difficulty >= 1
    prefix = "1" * difficulty
    for i in range(1000):
        digest = sha256(str(hash(message)) + str(i))
        if digest.startswith(prefix):
            print(f"after {str(i)} iterations found nonce: {digest}")
# return print(digest)
mine("test message", 2)

after 299 iterations found nonce: 1156f0731644a34afb03fca50218b97eeaca3689715b80577a938a5b5bffb63a
after 318 iterations found nonce: 11c21e901875899395035e8740d029e2ebfeff4d004195a3c3bb0958ef9975b2
after 411 iterations found nonce: 11cbc839bee7ab32bee8b6b73a7132d8f7300e5a4f48556787c20eed2fed5b2a
after 443 iterations found nonce: 1118a54bd5764d09c34e0dbc287747d62180e28ed0c2d1acca3e64961993bae4
after 449 iterations found nonce: 11c7fe9bce6ff740a796bafbc8a00a9b8a21c32dac797ebb8940d62b4befc853


In [21]:
#Add blocks to the miner and dump the blockchain.
import datetime
import hashlib
# Create a class with two functions
class Block:
    def __init__(self, data, previous_hash):
        self.timestamp = datetime.datetime.now(datetime.timezone.utc)
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.calc_hash()
    def calc_hash(self):
        sha = hashlib.sha256()
        hash_str = self.data.encode("utf-8")
        sha.update(hash_str)
        return sha.hexdigest()
# Instantiate the class
blockchain = [Block("First block", "0")]
blockchain.append(Block("Second block", blockchain[0].hash))
blockchain.append(Block("Third block", blockchain[1].hash))
# Dumping the blockchain
for block in blockchain:
    print(f"Timestamp: {block.timestamp}\n"
          f"Data: {block.data}\n"
          f"Previous Hash:{block.previous_hash}\n"
          f"Hash: {block.hash}\n"
)

Timestamp: 2024-04-01 02:50:33.826460+00:00
Data: First block
Previous Hash:0
Hash: 876fb923a443ba6afe5fb32dd79961e85be2b582cf74c233842b630ae16fe4d9

Timestamp: 2024-04-01 02:50:33.826460+00:00
Data: Second block
Previous Hash:876fb923a443ba6afe5fb32dd79961e85be2b582cf74c233842b630ae16fe4d9
Hash: 8e2fb9e02898feb024dff05ee0b27fd5ea0a448e252d975e6ec5f7b0a252a6cd

Timestamp: 2024-04-01 02:50:33.827500+00:00
Data: Third block
Previous Hash:8e2fb9e02898feb024dff05ee0b27fd5ea0a448e252d975e6ec5f7b0a252a6cd
Hash: 06e369fbfbe5362a8115a5c6f3e2d3ec7292cc4272052dcc3280898e3206208d



In [1]:
#Write a python program to implement mining using blockchain.
import hashlib
import time
class Block:
  def __init__(self, index, previous_hash, timestamp, data, hash):
    self.index = index
    self.previous_hash = previous_hash
    self.timestamp = timestamp
    self.data = data
    self.hash = hash
def calculate_hash(index, previous_hash, timestamp, data):
  value = str(index) + str(previous_hash) + str(timestamp) + str(data)
  return hashlib.sha256(value.encode()).hexdigest()
def create_genesis_block():
# Genesis block is the first block in the blockchain
  return Block(0, "0", time.time(), "Genesis Block", calculate_hash(0, "0", time.time(),"Genesis Block"))
def create_new_block(previous_block, data):
  index = previous_block.index + 1
  timestamp = time.time()
  hash = calculate_hash(index, previous_block.hash, timestamp, data)
  return Block(index, previous_block.hash, timestamp, data, hash)
def mine_block(previous_block, data, difficulty):
  new_block = create_new_block(previous_block, data)
  prefix = '0' * difficulty
  while new_block.hash[:difficulty] != prefix:
    new_block.timestamp = time.time()
    new_block.hash = calculate_hash(new_block.index, new_block.previous_hash,new_block.timestamp, new_block.data)
  print("Block mined:", new_block.hash)
  return new_block
if __name__ == "__main__":
  difficulty_level = 4 # Adjust the difficulty level as needed
  blockchain = [create_genesis_block()]
  previous_block = blockchain[0]
# Simulate mining multiple blocks
  for i in range(1, 5):
    new_data = f"Transaction data for Block {i}"
    previous_block = mine_block(previous_block, new_data, difficulty_level)
    blockchain.append(previous_block)

Block mined: 0000283ce1a0b430063da1cc881e48db1e77f81ae8ae5e10cfde079319c50138
Block mined: 00000d5bdddc80646fe84d5f23d1e6d0c859af4dde46d8ae538e18ed2193c749
Block mined: 00002fa626cc5b58365667cf0552266e9d3c8e902e9d5c965fab4eaa0a0f30c4
Block mined: 00006b5f050db3a71986438572994c580a151c8251a3b964c4aa3d05ecb5b944


In [ ]:
#Implement & demonstrate the use of the following solidity:
#Variables, operators, loop, decision making, strings, arrays, enum, structs, mapping, conversions, ether units, special variables.
pragma solidity ^0.8.0;
contract BasicSolidityFeatures {
// Variables
    uint public myUint = 10; 
    int public myInt = -5;
    bool public myBool = true;
// Enum
    enum Color {Red, Green, Blue} 
    Color public myColor = Color.Green;
// Struct
    struct Person { string name; uint age;
}
    Person public myPerson = Person("Alice", 30);
// Mapping
    mapping(uint => string) public idToName;
// Array
    uint[] public myArray = [1, 2, 3, 4, 5];
// Function to demonstrate operators and control flow 
    function arithmetic() public pure returns (uint) {
        uint a = 5; uint b = 3;
        uint result;
// Arithmetic operators 
        result = a + b; // Addition 
        result = a - b; // Subtraction
        result = a * b; // Multiplication
// If-else statement 
        if (a > b) {
            result = a;
        } else {
            result = b;
        }
// For loop
        for (uint i = 0; i < 5; i++) { 
            result += i;
        }
        return result;
    }
// Function to demonstrate strings
    function concatenateStrings() public pure returns (string memory) { 
        string memory str1 = "Hello";
        string memory str2 = " World";
        string memory result = string(abi.encodePacked(str1, str2)); 
        return result; // Returns "Hello World"
    }
// Function to demonstrate mappings
    function setName(uint id, string memory name) public { 
        idToName[id] = name;
    }
    function getName(uint id) public view returns (string memory) { 
        return idToName[id];
    }
// Function to demonstrate ether units
    function sendEther() public payable returns (uint) {
// Accepting ether
        uint amount = msg.value;
// Transferring ether
        payable(msg.sender).transfer(1 ether); // Transfer 1 ether
        return amount; // Returns the amount of ether received
}
}#id 101
#john

In [ ]:
#Implement & demonstrate the use of the following solidity Functions:
# a) Pure Function
# b) View Function
# c) Fall-back Function
# d) Function Overloading


# a) Pure Function

// SPDX-License-Identifier: MIT 
pragma solidity ^0.8.0;
contract MathOperations {
// Pure function to add two numbers
    function add(uint256 a, uint256 b) public pure returns (uint256) { 
        return a + b;
    }
// Pure function to subtract two numbers
    function subtract(uint256 a, uint256 b) public pure returns (uint256) { 
        require(a >= b, "Subtraction result cannot be negative.");
        return a - b;
}# add 7,5 Subtract 6,3
}

# b) View Function
pragma solidity ^0.8.0;
contract HelloWorld {
    string private greeting = "Hello, World!";
// View function to get the greeting message
function getGreeting() public view returns (string memory) { 
    return greeting;
}
}#hello world

# c) Fall-back Function
pragma solidity ^0.8.0;
    contract FallbackDemo {
        event Received(address sender, uint amount); 
        event FallbackCalled(address sender, uint amount);
// Fallback function 
        fallback() external payable {
            emit FallbackCalled(msg.sender, msg.value);
        }
// Receive Ether function 
        receive() external payable {
            emit Received(msg.sender, msg.value);
}
}

# d) Function Overloading

pragma solidity ^0.8.0; 
    contract MathOperations {
// Function to add two numbers
        function add(uint256 a, uint256 b) public pure returns (uint256) { 
            return a + b;
}
// Overloaded function to add three numbers
        function add(uint256 a, uint256 b, uint256 c) public pure returns (uint256) { 
            return a + b + c;
}
}

In [ ]:
#Write a smart contract to implement smart contract voting system.
pragma solidity ^0.8.0;
contract Poll {
    event Voted(address _voter, uint _value); 
    mapping(address => uint) public votes;
    string public pollSubject = "Press 1 for BJP or Press 2 for SHIVSENA";
    function getPoll() public view returns(string memory) { 
        return pollSubject;
}
    function vote(uint selection) public { 
        require(votes[msg.sender] == 0, "You have already voted.");
        require(selection > 0 && selection < 3, "Invalid selection. Press 1 for BJP or Press 2 for SHIVSENA");
        votes[msg.sender] = selection; 
        emit Voted(msg.sender, selection);
}
}